# In SVM2 Part


## Implement Multiclass Support Vector Machine  
    
    1) One-Versus-The rest approach
    2) One-Versus-One Approach
    

======================================================================================

### Importing Modules

In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC

C:\Users\jeonyunsu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Importing Datasets

In [2]:
frame="segmentation.csv"

data=pd.read_table(frame,sep=",")
X=data[['REGION-CENTROID-COL',"REGION-CENTROID-ROW","REGION-PIXEL-COUNT","SHORT-LINE-DENSITY-5","SHORT-LINE-DENSITY-2","VEDGE-MEAN","VEDGE-SD",
       "HEDGE-MEAN","HEDGE-SD","INTENSITY-MEAN","RAWRED-MEAN","RAWBLUE-MEAN","RAWGREEN-MEAN","EXRED-MEAN","EXBLUE-MEAN", "EXGREEN-MEAN","VALUE-MEAN","SATURATION-MEAN","HUE-MEAN"]]
Y=data['CLASSES']

CLASS=list(set(Y)) # one vs rest
cp_CLASS=list(set(Y)) # one vs one

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1,stratify=Y)


### Assign and Implment SVC Model

In [3]:
clf=SVC(kernel="linear")

- One VS The Rest

In [4]:
preds=[]

for one_vs_others in CLASS:
    clf.fit(X_train, np.where(one_vs_others==y_train,1,-1))
    pred_new_Y=[]
    for j in range(0,len(X_test)):
        values=sum(list(X_test.iloc[j,:])*clf.coef_[0])+clf.intercept_[0]
        pred_new_Y.append(values)
    preds.append(pred_new_Y)
    
    
final_Y=[CLASS[np.argmax(preds, axis=0)[i]] for i in range(0,len(y_test))]
acc1=sum(np.where(y_test==final_Y,1,0))/len(y_test)*100

- One VS One

In [5]:
preds2=[]

for one_vs in CLASS:
    cp_CLASS.remove(one_vs)
    for one in cp_CLASS:
        
        # model fit
        ix_modelset=y_train[y_train==one_vs].index.append(y_train[y_train==one].index)
        new_X=X_train.loc[list(ix_modelset),:]
        new_Y=y_train.loc[list(ix_modelset)]
        clf.fit(new_X, np.where(one_vs==new_Y,1,-1))
        
        # predict
        pred_Y=clf.predict(X_test)
        
        # index append
        preds2=preds2+list(np.where(pred_Y==1,one_vs,one))
        
## aggregate results
        
n=int((len(CLASS)*(len(CLASS)-1))/2)

final=pd.DataFrame(np.c_[preds2,pd.concat([y_test]*n),list(y_test.index)*n])
final.columns=['predict','real','index']
final=final.groupby(['index','predict','real']).size().reset_index(name="count")
final=final.iloc[final.groupby(['index'])['count'].idxmax()]
final['index']=final['index'].values.astype(int)
final=final.sort(['index'])

acc2=sum(np.where(final['real']==final['predict'],1,0))/len(final)*100

C:\Users\jeonyunsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Compare Accuracy

In [6]:
print (round(acc1,2),"%,", round(acc2,2),"%")

88.1 %, 90.48 %
